In [ ]:
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras import regularizers
from   tensorflow.keras import Sequential
from   tensorflow.keras.layers import Dropout, Dense

In [ ]:
from   matplotlib import pyplot as plt

In [ ]:
import errno
import glob
import json
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import subprocess
import tempfile
import uuid

In [ ]:
# Local modules
import config
import datasets
import op_stats
import utils

In [ ]:
# Globals
TICKER = 'SPY'

In [ ]:
# For saving the model
PREFIX = 'model'
TICKER_MODEL_DIR = os.path.join(config.ML_MODELS_DIR, TICKER)

In [ ]:
# Set the values to be used for working with the data
BATCH_SIZE     = 512
BUFFER_SIZE    = 100
MAX_MARGIN     = 10
MIN_PROFIT     = 1
MIN_DATAPOINTS = 4*10**6
MAX_EPOCHS     = 200

In [ ]:
try:
    os.makedirs(TICKER_MODEL_DIR)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
    for tmpdir in glob.glob('{}/{}*'.format(TICKER_MODEL_DIR, PREFIX)):
        shutil.rmtree(tmpdir)
MODEL_DIR = tempfile.mkdtemp(prefix='model', dir=TICKER_MODEL_DIR)

In [ ]:
# Load the data
ds = datasets.load_dataset(
    TICKER,
    max_margin=MAX_MARGIN,
    min_profit=MIN_PROFIT,
    total_datapoints=MIN_DATAPOINTS,
    loss_ratio=3,
    loss_pool_multiplier=2,
    verbose=True,
)

metadata = ds.metadata
# Are we using the right ticker?
assert(metadata['ticker'] == TICKER)
# Make sure we're at least reasonably close to the desired number of datapoints
assert(metadata['total_datapoints'] >= MIN_DATAPOINTS*0.99)
# Make sure the remaining metadata is accurate
assert(metadata['max_margin'] == MAX_MARGIN)
assert(metadata['min_profit'] == MIN_PROFIT)

STEPS_PER_EPOCH = ds.n_train//BATCH_SIZE

In [ ]:
# Collect the statistics
pooled_means, pooled_variances = op_stats.pool_stats_from_stats_df(TICKER)

In [ ]:
# Immediately save the metadata that we can
pooled_means.to_pickle(
    os.path.join(MODEL_DIR, 'means'),
    protocol=config.PICKLE_PROTOCOL
)
pooled_variances.to_pickle(
    os.path.join(MODEL_DIR, 'variances'),
    protocol=config.PICKLE_PROTOCOL
)
pooled_stds = pooled_variances.pow(1/2)

In [ ]:
pooled_means

In [ ]:
# It'll be important for code using this model to know how to order its
# columns.
feature_order = pooled_means.index.tolist()
n_features = len(feature_order)
print('{} features'.format(n_features))
metadata['feature_order'] = feature_order
print(feature_order)

In [ ]:
# Load and set up the datasets for train, validation and test
(train_dataset,
 validate_dataset,
 test_dataset) = ds.get_sets(feature_order, pooled_means, pooled_stds)

train_dataset    = train_dataset.shuffle(ds.n_train)
train_dataset    = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).repeat()
validate_dataset = validate_dataset.shuffle(ds.n_val).batch(BATCH_SIZE)
test_dataset     = test_dataset.shuffle(ds.n_test).batch(BATCH_SIZE)

In [ ]:
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
    0.00001,
    decay_steps=STEPS_PER_EPOCH,
    decay_rate=1,
    staircase=False
)

checkpoint_filepath = os.path.join(MODEL_DIR, 'checkpoint')
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

class MetadataSaver(keras.callbacks.Callback):
    _best_loss = np.inf
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_loss'] >= self._best_loss:
            return
        
        self._best_loss = logs['val_loss']
        with open(os.path.join(MODEL_DIR, 'metadata'), 'w') as MF:
            metadata.update({
                'accuracy': float(logs['val_accuracy']),
                'loss': float(self._best_loss),
            })
            json.dump(metadata, MF)

def get_callbacks(name):
    return [
        # tfdocs.modeling.EpochDots(),
        model_checkpoint_callback,
        MetadataSaver(),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    ]

model = Sequential([
    Dense(32, activation='relu', input_shape=(n_features,)),
    Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(lr_schedule),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(train_dataset,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=MAX_EPOCHS,
                    validation_data=validate_dataset,
                    callbacks=get_callbacks('testing'),
                    verbose=1)

In [ ]:
# Get the testing data in numpy form
test_Xs = []
test_Ys = []
for x, y in test_dataset.as_numpy_iterator():
    test_Xs.append(x)
    test_Ys.append(y)
test_Xs = np.concatenate(test_Xs)
test_Ys = np.concatenate(test_Ys)

In [ ]:
# Make some predictions
preds = model.predict(test_Xs)[:,0]

In [ ]:
# Figure out what the percentiles are for the predictions
percentiles = list(range(50, 100, 5)) + [99, 99.9, 99.99, 99.999]
percentiles = dict(zip(percentiles, np.percentile(preds, percentiles)))

In [ ]:
# Get the Precision, Recall and F1 score for the predictions
pred_trues = preds >= 0
true_pos = pred_trues & test_Ys
precision = true_pos.sum() / pred_trues.sum()
recall = true_pos.sum() / test_Ys.sum()
f1_score = 2 * precision * recall / (precision + recall)

In [ ]:
with open(os.path.join(MODEL_DIR, 'metadata'), 'r+') as MF:
    metadata = json.load(MF)
    MF.seek(0)
    metadata.update({
        'percentiles': percentiles,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
    })
    json.dump(metadata, MF)
    MF.truncate()

In [ ]:
# Build a tarball for this session (ignoring the directory when including filenames)
files_to_tar = [os.path.basename(f) for f in glob.glob('{}/*'.format(MODEL_DIR))]
tarball_path = os.path.join(config.ML_MODELS_DIR, TICKER, '{}.tar'.format(uuid.uuid4()))
subprocess.check_call(['tar', '-C', MODEL_DIR, '-cf', tarball_path] + files_to_tar)
shutil.rmtree(MODEL_DIR, ignore_errors=True)

In [ ]:
# Test the model
model.evaluate(test_dataset)

In [ ]:
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
# plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()